# 1. Giới thiệu:
MobileNet là một họ kiến trúc CNN được phát triển bởi Google, với mục tiêu tối ưu hóa cho các thiết bị di động và hệ thống có tài nguyên hạn chế như điện thoại thông minh, thiết bị IoT hay vi xử lý nhúng. Ý tưởng là tách các kênh trong 1 ảnh ra, học riêng các feature trong từng kênh, sau đó mới gộp lại. Các mô hình trong họ MobileNet giúp giảm thiểu số lượng parameters và lượng phép tính (FLOPs), đồng thời vẫn duy trì độ chính xác cao trong các bài toán thị giác máy tính.

Các ứng dụng phổ biến:
- Phân loại hình ảnh
- Nhận diện đối tượng
- Nhận diện khuôn mặt
- Phát hiện tư thế
- Sử dụng trong các framework như: TensorFlow Lite, CoreML, ONNX,...

# 2. Vấn đề:
Trong CNN truyền thống, một lớp convolution chuẩn sử dụng một tập kernel 3×3 (hoặc 5×5) để trích xuất đặc trưng từ toàn bộ ảnh đầu vào, đồng thời trộn thông tin giữa các kênh màu để tạo ra một đặc trưng tổng hợp. Cách hoạt động của nó gây ra 3 vấn đề chính:

### 1. FLOPs:
FLOPs cho 1 lớp convolution truyền thống:
$$FLOPs=H*W*K^2*M*N$$
Với:
- M: số kênh đầu vào
- N: số kênh đầu ra
- K: kích thước kernel (thường là 3)
- H × W: kích thước không gian đầu vào

Điều này rất tốn kém, đặc biệt khi số kênh lớn. Lấy ví dụ 1 ảnh đầu vào 224x224x3, N = 64, K = 3:

$$FLOPs=224*224*3^2*3*64 \approx 86000000$$

### 2. Parameters:
Số parameter cũng bị ảnh hưởng nghiêm trọng. Ta có công thức:
$$\text{layer param} = K^2*M*N$$

Ví dụ với $K=3, M=128, N=256$, ta đã có:
$$\text{layer param} = 3^2*128*256 = 294912$$

Đây là số lượng tham số chỉ cho 1 lớp CNN. Vấn đề này khiến bộ nhớ và thời gian tải mô hình tăng theo.

### 3. Ép 1 bộ lọc làm 2 nhiệm vụ cùng lúc:
Khi ta áp dụng cùng 1 kernel cho toàn bộ không gian ảnh, bộ lọc phải làm cùng lúc 2 việc:
- Học cách trích xuất các feature không gian (edge, texture,...)
- Học cách kết hợp thông tin giữa các kênh màu

Việc này khi tách ra sẽ giảm rất nhiều độ phức tạp tính toán.

# 3. Giải pháp đề xuất:
### 1. Intuition đằng sau giải pháp: 
Không phải phép conv nào cũng cần học quan hệ giữa các kênh cùng 1 lúc. Các feature trong những kênh riêng biệt, rồi mới kết hợp lại như trong xử lý ảnh cổ điển (dùng kernel như Sobel để lọc riêng từng kênh, sau đó mới gộp thông tin)
### 2. Depthwise seperable convolution:
Thay vì dùng 1 bước convolution 3D thông thường ($H*W*N$), ta thay bằng 2 bước riêng biệt:
- Depthwise convolution (lọc không gian):
    - 1 filter 2D (thường là $3*3$) được áp dụng cho MỖI KÊNH đầu vào.
    - Không làm thay đổi số kênh, chỉ áp dụng filter cho từng kênh.
    
    VD: ảnh inp có 3 kênh màu -> tensor vẫn có 3 kênh nhờ áp dụng kernel $3*3$ cho mỗi kênh input.
- Pointwise convolution (kết hợp kênh):
    - Dùng filter $1*1*K_{input}$ để kết hợp các giá trị tại cùng vị trí không gian nhưng trên nhiều kênh khác nhau.
    - Đây là giai đoạn mô hình học mối quan hệ giữa các kênh như conv bình thường. 
    
    VD: 3 kênh đầu vào (sau deptwise conv) -> 64 kênh output bằng cách dùng 64 kernel $1*1*3$




